# Recurrent Neural Nets in Tensorflow

Authors: Jeremy Locatelli, Erik Ring-Walters

## Introduction

In this project, we aimed to create a recurrent neural net in TensorFlow based on Andrej Karpathy's code in plain Python. This net reads a set of characters (Alice in Wonderland, for example) and generates text that aims to look closer to the original text.

Here we test it on a smaller dataset, but one with a much different style, every line from the hit moviestaring Jerry Seinfeld, The Bee Movie. 

In [1]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [2]:
data = open('./jerry.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

seq_length = 25

X = []
y = []

data has 769554 characters, 86 unique.


In [3]:
for i in range(0, len(data)-seq_length-1, 1):
        X.append([char_to_ix[ch] for ch in data[i:i+seq_length]])
        y.append([char_to_ix[ch] for ch in data[i+1:i+seq_length+1]])

# reshape the data
# in X_modified, each row is an encoded sequence of characters
X_modified = np.reshape(X, (len(X), seq_length))
y_modified = np.reshape(y, (len(y), seq_length))

In [4]:
# @params: X: the input array to fetch a batch from
# @params: y: the output array to fetch a corosponding batch from
# @params: batch_size: The size of the batch you want to pull
def fetchBatch(X,y,batch_size):
    idx = np.random.randint((len(X)-batch_size)-1)
    X_batch = X[idx:idx+batch_size,:]
    y_batch = y[idx:idx+batch_size,:]
    yield X_batch, y_batch

In [5]:
# Pulls a single random row from X_modifed
def single_batch(X):
    rnd = np.random.randint(len(X)-1)
    return X[rnd]

In [6]:
   
n_inputs = vocab_size
n_outputs = vocab_size
n_steps = 25
n_neurons = 100
tf.reset_default_graph()

## First problem we faced
one of the first big design decisions we made was whether to one hot encode each batch in tensorflow, or one hot encode the entire input text, then create batches with that. We decided to one hot encode each batch because we had trouble creating a fetch batch function on the large one hot encoded 3d array.

In [7]:
X = tf.placeholder(tf.int32, [None, n_steps])
one_hot_X = tf.one_hot(X, vocab_size)
y = tf.placeholder(tf.int32, [None, n_steps])
cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicRNNCell(
        num_units=n_neurons, activation=tf.nn.relu),output_size=n_outputs)

cpu_gru_cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.GRUBlockCellV2(
        num_units=n_neurons),output_size=n_outputs)

outputs, states = tf.nn.dynamic_rnn(cell, one_hot_X, dtype=tf.float32) 
learning_rate = 0.001
y_probs = tf.nn.softmax(logits=outputs)
xenthropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=outputs, labels=y)
loss = tf.reduce_mean(xenthropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)


## Sampling data
#### Thanks Dr. Bruns
This was the biggest breakthrough in terms of understanding, creating a "floating window" that tracked the updated sequence we feed to the RNN. Once we got this working it led to the discovery of several other issues. 
One was for some reason all out our outputs were converging towards a single character, e.g. i would call sample with length 100 and it would spit out 100 commas (or newline characters which does wonders for a console program..)
I tried to solve this bug for hours, looking at the probability distributions for sequential predictions. till i discovered. In my fetch batch function, i accidentaly set both X and y_batch to be the same thing... Once i fixed that one character error, things started to improve. 

In [8]:
def sample(seed_pred, n):
    start_idx = 0
    # list of indexes we predict
    output = []
    # Debug for seeing the prob distributions
    probs_list = []
    
    for i in range(n):
      # This creates the moving window that starts with our input sequence, and the new input is appended at each loop
      window = seed_pred[start_idx:start_idx + n_steps]
      # We then rotate it to batch_size = 1 and 25 cols for entry into the X tensor
      window = window.reshape((1, n_steps))
      # We evaluate the softmax of the outputs by inserting our array window into the network and getting the logits
      char_prob = y_probs.eval(feed_dict={X:window})
      probs_list.append(char_prob)
      # We only care about the last character softmaxed outputs, as this is the probability of a choice of one of len(vocab_size) characters
      last_char = char_prob[0][n_steps-1]
      # We get an index value by pulling a random value /according to the probabilty distribution of the last_chars softmax      
      ind = np.random.choice(range(vocab_size), p=last_char.ravel())
      # tack that onto our big list
      output.append(ind)
      # reshape ind so we can tack it to the bottom of seed_pred
      ind = ind.reshape((1,))
      # append it to seed_pred
      seed_pred = np.append(seed_pred, ind, axis=0)
      # move the window up one and continue
      start_idx +=1
    return output, probs_list

In [9]:
init = tf.global_variables_initializer()

n_epochs = 100000
batch_size = 50
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in fetchBatch(X_modified,y_modified,batch_size):
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            if epoch % 10000 == 0:
                cur_loss = loss.eval(feed_dict={X:X_batch, y:y_batch})
                print(f"Current epoch: {epoch} \tLoss: \t{cur_loss}\n")
                solo_batch = single_batch(X_modified)
                smpl, _ = sample(solo_batch,100)
                txt = ''.join(ix_to_char[ix] for ix in smpl)
                print("OUTPUT")
                print('===============\n %s \n===============' % (txt, ))
                
                    

  

Current epoch: 0 	Loss: 	4.439173221588135

OUTPUT
 wFa0vxUNJ5o-toOKgiU?e#HRAt;V!QF1eE849h6p]Dh(`XR6wc2CWFofh-keV!-?AEt-omz<_9A\k!6-Hc\;6]%tJOEu!F"!3fRF 


KeyboardInterrupt: 

# GRU RNN network
Here we build a RNN with the exact same architecture as the last one, but we use GRU Cells instead of basic RNN cells.

In [11]:
tf.reset_default_graph() 
X = tf.placeholder(tf.int32, [None, n_steps])
one_hot_X = tf.one_hot(X, vocab_size)
y = tf.placeholder(tf.int32, [None, n_steps])

cpu_gru_cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.GRUBlockCellV2(
        num_units=n_neurons),output_size=n_outputs)

outputs, states = tf.nn.dynamic_rnn(cpu_gru_cell, one_hot_X, dtype=tf.float32) 
learning_rate = 0.001
y_probs = tf.nn.softmax(logits=outputs)
xenthropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=outputs, labels=y)
loss = tf.reduce_mean(xenthropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

In [12]:
init = tf.global_variables_initializer()

n_epochs = 100000
batch_size = 50
with tf.Session() as sess:
    init.run()
    print("===GRU RNN===")
    for epoch in range(n_epochs):
        for X_batch, y_batch in fetchBatch(X_modified,y_modified,batch_size):
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            if epoch % 10000 == 0:
                cur_loss = loss.eval(feed_dict={X:X_batch, y:y_batch})
                print(f"Current epoch: {epoch} \tLoss: \t{cur_loss}\n")
                solo_batch = single_batch(X_modified)
                smpl, _ = sample(solo_batch,100)
                txt = ''.join(ix_to_char[ix] for ix in smpl)
                print("OUTPUT")
                print('===============\n %s \n===============' % (txt, ))
                
                    

===GRU RNN===
Current epoch: 0 	Loss: 	4.440817356109619

OUTPUT
 Yq<]?Ql8vMnS8g?fRq/ZNv'GI#;mJ9JgI0hKLwFWw;,O5;JZp_JyCY"5n<#4NFuedsxcK/azOVxl$1%[3srdND]Ued4l%s/wkO0f 
Current epoch: 10000 	Loss: 	1.802346110343933

OUTPUT
 morcmeca verys is shee tar-Misper) There whing that! I do do it like.Yeah, I gonna businna up of a I 


KeyboardInterrupt: 

# ERIK MAKE ANOTHER NETWORK THATS THE SAME AS ABOVE BUT FOR LSTM ALSO DELETE THIS BEFORE TURN IN LOL
then make a conclusion